In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string
import os
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import warnings 
warnings.filterwarnings('ignore') 
#nltk.download('wordnet')
#nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

In [2]:
model_ner = spacy.load('./model-best/')

## Parsing / Extract Skill

In [3]:
def merge_tokens(df):
    merged_tokens = []
    current_token = ''
    current_label = ''
    
    for i, row in df.iterrows():
        token = lemmatizer.lemmatize(row['tokens'].lower())
        label = row['label']
        
        if label.startswith('B-'):
            if current_token:
                merged_tokens.append((current_token, current_label))
            current_token = token
            current_label = label
        elif label.startswith('I-') and current_label[2:] == label[2:]:
            current_token += ' ' + token
        else:
            current_token = ''
            current_label = ''
    
    if current_token:
        merged_tokens.append((current_token, current_label))

    merged_tokens = list(set(merged_tokens))
    
    return merged_tokens

In [4]:
def parsing_ner(text):
    doc = model_ner(text)
    doc_json = doc.to_json()
    doc_text = doc_json['text']
    
    df_tokens = pd.DataFrame(doc_json['tokens'])
    df_tokens['tokens'] = df_tokens[['start', 'end']].apply(
        lambda x: doc_text[x[0]: x[1]], axis=1
    )
    
    if not doc_json['ents']:
        return {}
    
    right_table = pd.DataFrame(doc_json['ents'])[['start', 'label']]
    df_tokens = pd.merge(df_tokens, right_table, how='left', on='start')
    
    df_tokens = df_tokens.query('label == label')
    merged = merge_tokens(df_tokens)
    print(df_tokens) 
    
    merged_df = pd.DataFrame(merged, columns=['tokens', 'label'])
    result = merged_df.groupby('label')['tokens'].apply(list).to_dict()
    
    return result


## Import data and Data preparation

In [5]:
job_desc = pd.read_csv('./Data/job_listings_DA.csv')
job_desc.head()

,Title,Company,Description
0,Data Analyst,Yayasan Hutan Tropis,"Data Analyst: Collect, clean and analyze infor..."
1,DATA ANALYST,PT Karya Anugerah Jaya,KUALIFIKASI :: DESKRIPSI PEKERJAAN :: NOTE : P...
2,Data Analyst Jr. Specialist,PT Bussan Auto Finance (BAF),Job Description : : Qualification : : Job Desc...
3,Data Analyst,"PT SMART,Tbk",Job Description Summary:: Job Description:: Jo...
4,Data Analyst - ERP Staff (JAKARTA),PT Allure Alluminio,ERP(Dynamics 365 Business Central): Bersedia d...


In [6]:
results = job_desc['Description'].apply(parsing_ner)
results_df = pd.DataFrame(results.tolist(), index=job_desc.index)
job_desc = job_desc.join(results_df)
job_desc = job_desc.fillna(0)

for column in results_df.columns:
   job_desc[column] = job_desc[column]

print(job_desc.head())

      id  start   end   dep  head           tokens     label
8      8     41    52  ROOT     8      information  I-HSkill
19    19    113   122  ROOT    19        processes  I-HSkill
28    28    163   165  ROOT    28               of  B-HSkill
61    61    335   345  ROOT    61       ecosystems  I-HSkill
139  139    748   755  ROOT   139          complex  B-HSkill
140  140    756   760  ROOT   140             data  I-HSkill
141  141    761   765  ROOT   141             sets    I-Tech
156  156    854   863  ROOT   156        expertise  I-HSkill
158  158    867   872  ROOT   158            Excel    B-Tech
164  164    903   909  ROOT   164           macros  I-HSkill
166  166    914   917  ROOT   166              VBA    B-Tech
168  168    919   923  ROOT   168             will  B-HSkill
174  174    955   965  ROOT   174       optimizing  B-HSkill
176  176    970   974  ROOT   176             data  I-HSkill
177  177    975   984  ROOT   177        processes  I-HSkill
186  186   1024  1034  R

      id  start   end   dep  head          tokens     label
4      4     33    43  ROOT     4      understand  B-HSkill
29    29    180   188  ROOT    29        analysis  I-HSkill
31    31    190   204  ROOT    31  classification  B-HSkill
34    34    210   220  ROOT    34      clustering  I-HSkill
40    40    249   257  ROOT    40        analysis  I-HSkill
42    42    262   269  ROOT    42         feature  B-HSkill
43    43    270   281  ROOT    43     engineering  I-HSkill
47    47    304   312  ROOT    47        insights  I-HSkill
55    55    351   362  ROOT    55     statistical  B-HSkill
56    56    363   370  ROOT    56         methods  I-HSkill
60    60    392   402  ROOT    60      algorithms  I-HSkill
63    63    413   422  ROOT    63       effective  I-HSkill
64    64    423   427  ROOT    64            data  I-HSkill
70    70    456   460  ROOT    70            data  I-HSkill
71    71    461   475  ROOT    71  visualizations  I-HSkill
75    75    486   496  ROOT    75      d

      id  start   end   dep  head         tokens     label
5      5     23    30  ROOT     5        healthy  B-HSkill
20    20    103   110  ROOT    20        comment    B-Tech
26    26    141   142  ROOT    26              @    I-Tech
66    66    363   370  ROOT    66        healthy  B-HSkill
73    73    401   409  ROOT    73       products  I-HSkill
84    84    455   461  ROOT    84         Europe    B-Tech
107  107    615   622  ROOT   107        focused  I-HSkill
115  115    685   695  ROOT   115     technology  I-HSkill
127  127    756   766  ROOT   127     production    B-Tech
133  133    799   811  ROOT   133   technologies  I-HSkill
177  177   1046  1054  ROOT   177       products  I-HSkill
208  208   1248  1260  ROOT   208   Consultation  I-HSkill
214  214   1283  1293  ROOT   214     experience  I-HSkill
221  221   1328  1335  ROOT   221        provide  B-HSkill
236  236   1462  1474  ROOT   236   Descriptions  I-HSkill
242  242   1504  1508  ROOT   242           data  I-HSki

      id  start   end   dep  head       tokens     label
5      5     26    37  ROOT     5  pembersihan  B-SSkill
9      9     54    58  ROOT     9         data  I-HSkill
30    30    219   223  ROOT    30         data  I-HSkill
52    52    386   391  ROOT    52        hasil    B-Tech
58    58    437   440  ROOT    58          tim  B-SSkill
95    95    690   694  ROOT    95         Data  B-HSkill
119  119    852   856  ROOT   119         data  B-HSkill
125  125    886   892  ROOT   125       Python    B-Tech
129  129    895   899  ROOT   129         SPSS    B-Tech
148  148   1032  1042  ROOT   148   komunikasi  B-SSkill
150  150   1047  1057  ROOT   150   presentasi  B-SSkill
168  168   1174  1181  ROOT   168      masalah  I-SSkill
173  173   1214  1218  ROOT   173         data  I-HSkill
      id  start   end   dep  head        tokens     label
17    17    103   109  ROOT    17        sistem  B-HSkill
18    18    110   118  ROOT    18      database  I-HSkill
80    80    536   544  ROOT 

      id  start   end   dep  head         tokens     label
17    17     78    82  ROOT    17           Data  I-HSkill
21    21     94   101  ROOT    21        Invoice    I-Tech
34    34    161   168  ROOT    34        Digital  B-HSkill
35    35    169   176  ROOT    35        Process  I-HSkill
44    44    219   223  ROOT    44           Data    B-Tech
45    45    224   235  ROOT    45    Preparation  I-HSkill
73    73    382   390  ROOT    73       enabling  B-HSkill
90    90    501   514  ROOT    90  documentation  B-HSkill
105  105    610   621  ROOT   105    operational  I-HSkill
115  115    681   692  ROOT   115    initiatives  I-HSkill
151  151    939   947  ROOT   151       Document  I-HSkill
166  166   1046  1050  ROOT   166           team  B-SSkill
173  173   1077  1084  ROOT   173        Prepare    B-Tech
182  182   1122  1130  ROOT   182       database  I-HSkill
192  192   1199  1204  ROOT   192          tasks  I-HSkill
195  195   1208  1212  ROOT   195           Data  B-HSki

...

      id  start   end   dep  head         tokens     label
15    15    101   106  ROOT    15          jawab  I-SSkill
19    19    131   135  ROOT    19           data  I-HSkill
27    27    193   200  ROOT    27        masalah  I-SSkill
106  106    701   714  ROOT   106  berkomunikasi  B-SSkill
113  113    760   766  ROOT   113         Bahasa  B-SSkill
114  114    767   774  ROOT   114        Inggris  I-SSkill
132  132    861   872  ROOT   132    Programming  B-HSkill
137  137    891   894  ROOT   137            ERP    B-Tech
140  140    907   915  ROOT   140       Business  B-HSkill
141  141    916   923  ROOT   141        Process  I-HSkill
149  149    971   978  ROOT   149        Project  B-HSkill
151  151    990   998  ROOT   151       software  I-HSkill
153  153   1000  1003  ROOT   153            SAP    B-Tech
162  162   1028  1036  ROOT   162       Internet  B-HSkill
170  170   1082  1093  ROOT   170    pemrograman  B-HSkill
171  171   1094  1095  ROOT   171              C    B-Te

      id  start   end   dep  head        tokens     label
46    46    266   270  ROOT    46          Odoo  I-HSkill
51    51    292   301  ROOT    51     Framework    B-Tech
71    71    402   410  ROOT    71      Database  B-HSkill
72    72    411   421  ROOT    72    PostgreSQL    B-Tech
80    80    457   462  ROOT    80         rajin  B-SSkill
97    97    546   551  ROOT    97         modul  I-HSkill
116  116    667   672  ROOT   116         modul  I-HSkill
123  123    720   723  ROOT   123           tim  B-SSkill
127  127    748   757  ROOT   127     knowledge  I-HSkill
134  134    806   815  ROOT   134     penerapan  B-HSkill
135  135    816   822  ROOT   135        sistem  I-HSkill
142  142    860   864  ROOT   142          Odoo    B-Tech
148  148    908   915  ROOT   148       masalah  I-SSkill
162  162    996  1006  ROOT   162    programmer  B-HSkill
166  166   1026  1038  ROOT   166  pengembangan  B-HSkill
167  167   1039  1045  ROOT   167        sistem  I-HSkill
      id  star

      id  start   end   dep  head          tokens     label
4      4     20    24  ROOT     4            will  I-HSkill
21    21     96   100  ROOT    21            will  I-HSkill
33    33    152   163  ROOT    33     Responsible  B-HSkill
34    34    164   167  ROOT    34             for  I-HSkill
35    35    168   176  ROOT    35        planning  I-HSkill
68    68    355   362  ROOT    68         mission  B-HSkill
71    71    368   378  ROOT    71      objectives  B-SSkill
85    85    447   455  ROOT    85        planning  I-HSkill
89    89    468   478  ROOT    89      monitoring  B-HSkill
90    90    479   489  ROOT    90      evaluation  I-HSkill
104  104    582   593  ROOT   104     preparation  I-HSkill
109  109    623   630  ROOT   109         process  I-HSkill
119  119    690   704  ROOT   119  recommendation  I-HSkill
199  199   1179  1187  ROOT   199        Security  B-HSkill
203  203   1203  1210  ROOT   203         Working  B-HSkill
220  220   1305  1315  ROOT   220      e

      id  start   end   dep  head       tokens     label
17    17    105   111  ROOT    17       System  I-HSkill
19    19    113   124  ROOT    19  Informatics  B-HSkill
20    20    125   136  ROOT    20  Engineering  I-HSkill
35    35    201   211  ROOT    35   processing  I-HSkill
63    63    351   359  ROOT    63     cleaning  I-HSkill
66    66    375   380  ROOT    66        using  I-HSkill
69    69    386   394  ROOT    69     software  B-HSkill
74    74    407   414  ROOT    74      bespoke  I-HSkill
76    76    423   431  ROOT    76     analyses  I-HSkill
88    88    496   504  ROOT    88     requests  I-HSkill
92    92    522   531  ROOT    92    delivered  I-HSkill
97    97    543   547  ROOT    97         Data  B-HSkill
112  112    628   634  ROOT   112       System  I-HSkill
114  114    636   647  ROOT   114  Informatics  B-HSkill
115  115    648   659  ROOT   115  Engineering  I-HSkill
124  124    698   707  ROOT   124    Microsoft    B-Tech
125  125    708   713  ROOT   1

      id  start   end   dep  head        tokens     label
0      0      0     3  ROOT     0           Job  B-HSkill
51    51    267   270  ROOT    51           Why    B-Tech
62    62    325   337  ROOT    62  deliverables  I-HSkill
83    83    469   471  ROOT    83            to  I-HSkill
85    85    480   483  ROOT    85           our  I-HSkill
86    86    484   491  ROOT    86       project  B-HSkill
87    87    492   500  ROOT    87      delivery  I-HSkill
91    91    524   533  ROOT    91     processes  I-HSkill
114  114    687   696  ROOT   114     materials  I-HSkill
121  121    732   735  ROOT   121           our  I-HSkill
144  144    864   866  ROOT   144            of  B-HSkill
146  146    879   889  ROOT   146    technology  I-HSkill
148  148    891   901  ROOT   148    accounting  B-HSkill
155  155    933   941  ROOT   155      relevant  B-HSkill
166  166    982   989  ROOT   166       working  B-HSkill
173  173   1022  1025  ROOT   173           use  I-HSkill
177  177   104

In [7]:
results_df

,B-HSkill,B-SSkill,B-Tech
0,"[dynamic environment information innovate, our...",[bahasa],"[palm, vba, soy]"
1,"[familiar, data analyst data, accurate, program]","[jujur, interpersonal, teliti, komunikasi, ber...","[microsoft office, note]"
2,"[job data, tool]","[tim, komunikasi]","[excel, sql]"
3,"[supplier desktop, reliable verification descr...",NaN,NaN
4,"[mengolah, analisis]","[mandiri, tim, komunikasi]",NaN
...,...,...,...
123,"[information technology field, database manage...",NaN,NaN
124,[database],"[kepemimpinan, komunikasi, organisasi]",NaN
125,[data performance],NaN,NaN
126,NaN,[konsultasi jawabnya],NaN


In [8]:
job_desc.head()

,Title,Company,Description,B-HSkill,B-SSkill,B-Tech
0,Data Analyst,Yayasan Hutan Tropis,"Data Analyst: Collect, clean and analyze infor...","[dynamic environment information innovate, our...",[bahasa],"[palm, vba, soy]"
1,DATA ANALYST,PT Karya Anugerah Jaya,KUALIFIKASI :: DESKRIPSI PEKERJAAN :: NOTE : P...,"[familiar, data analyst data, accurate, program]","[jujur, interpersonal, teliti, komunikasi, ber...","[microsoft office, note]"
2,Data Analyst Jr. Specialist,PT Bussan Auto Finance (BAF),Job Description : : Qualification : : Job Desc...,"[job data, tool]","[tim, komunikasi]","[excel, sql]"
3,Data Analyst,"PT SMART,Tbk",Job Description Summary:: Job Description:: Jo...,"[supplier desktop, reliable verification descr...",0,0
4,Data Analyst - ERP Staff (JAKARTA),PT Allure Alluminio,ERP(Dynamics 365 Business Central): Bersedia d...,"[mengolah, analisis]","[mandiri, tim, komunikasi]",0


In [9]:
import spacy
from spacy import displacy

item = job_desc.iloc[0]['Description']  

doc = model_ner(item)
displacy.render(doc, style="ent", jupyter=True)

In [10]:
results_df

,B-HSkill,B-SSkill,B-Tech
0,"[dynamic environment information innovate, our...",[bahasa],"[palm, vba, soy]"
1,"[familiar, data analyst data, accurate, program]","[jujur, interpersonal, teliti, komunikasi, ber...","[microsoft office, note]"
2,"[job data, tool]","[tim, komunikasi]","[excel, sql]"
3,"[supplier desktop, reliable verification descr...",NaN,NaN
4,"[mengolah, analisis]","[mandiri, tim, komunikasi]",NaN
...,...,...,...
123,"[information technology field, database manage...",NaN,NaN
124,[database],"[kepemimpinan, komunikasi, organisasi]",NaN
125,[data performance],NaN,NaN
126,NaN,[konsultasi jawabnya],NaN


In [11]:
results_df.to_excel('./Data/skills_DA.xlsx', index=False, engine='openpyxl')

In [11]:
skill_list = results_df['B-Tech'].explode().tolist()

In [12]:
skill_list2 = results_df['B-Tech'].explode().tolist()

In [14]:
len(skill_list2)

582

In [15]:
skill_list = list(set(skill_list))

In [16]:
#exp = pd.DataFrame(skill_list2, columns=['Skills'])  # Create a DataFrame
#exp.to_excel('skill_list2.xlsx', index=False)  # Export to Excel

Manually Edit the list !

In [17]:
data = pd.read_excel('./Data/skill_list.xlsx')

In [18]:
data

,Skills
0,db2
1,google cloud
2,job etl
3,unix
4,microsoft excel
...,...
188,exploratory data analysis
189,artificial intelligence
190,ci cd
191,google cloud platform


In [14]:
from collections import Counter

counts = Counter(skill_list2)
df = pd.DataFrame(counts.items(), columns=['Item', 'Count'])
df = df.dropna()

In [15]:
df.sort_values('Count', ascending =False).head(50)

,Item,Count
6,sql,19
5,excel,17
18,python,12
66,written,6
57,microsoft,6
3,microsoft office,5
8,microsoft excel,5
26,etl,4
92,sql server,4
61,key,4


In [21]:
#df.to_excel('./Data/skill_frequency.xlsx', index=False) 

In [22]:
import re
pattern = r'(\d+)[\s-]*(\d*)\s*(?:years?|tahun|th|yrs?|experience|pengalaman)'

def extract_experience(text):
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        if match.group(2):
            return f"{match.group(1)}-{match.group(2)} years"
        else:
            return f"{match.group(1)} years"
    else:
        if re.search(r'fresh\s*graduate', text, re.IGNORECASE):
            return "Fresh Graduate"
        return None

In [23]:
job_desc['Experience Required'] = job_desc['Description'].apply(extract_experience)

In [24]:
job_desc['Experience Required'].unique()

array(['3-5 years', None, '30 years', '45 years', '25-35 years',
       '2 years', '3 years', '5 years', '35 years', '4 years',
       'Fresh Graduate', '2-5 years', '28 years', '1 years', '1-2 years',
       '2-3 years', '20-30 years', '1-3 years', '15 years', '23-35 years',
       '26 years', '27 years', '5-8 years', '25 years'], dtype=object)

In [25]:
unique_counts = job_desc['Experience Required'].value_counts()
unique_counts

2 years           15
1 years           12
3 years            8
5 years            6
2-3 years          5
4 years            5
3-5 years          4
Fresh Graduate     4
30 years           3
1-2 years          3
35 years           3
28 years           2
25 years           2
2-5 years          1
25-35 years        1
45 years           1
20-30 years        1
1-3 years          1
15 years           1
23-35 years        1
26 years           1
27 years           1
5-8 years          1
Name: Experience Required, dtype: int64

In [26]:
def normalize_experience(experience):
    if experience is None:  # Check for None
        return np.nan
    if 'Fresh Graduate' in experience:
        return 0
    elif 'year' in experience:
        numbers = re.findall(r'\d+', experience)
        if len(numbers) == 1:
            return int(numbers[0])
        elif len(numbers) == 2:
            return (int(numbers[0]) + int(numbers[1])) / 2  # Average of the range
    return np.nan  # Return NaN for unexpected formats

In [27]:
job_desc['Normalized Experience'] = job_desc['Experience Required'].apply(normalize_experience)

In [28]:
bins = [-1, 0, 1, 2, 3, 5, np.inf]  # Adjust as needed
labels = ['Fresh Graduate', '1 Year', '1 - 2 Years', '2 - 3 Years', '3 - 5 Years', '5+ Years']

# Create a new column for binned categories
job_desc['Experience Category'] = pd.cut(job_desc['Normalized Experience'], bins=bins, labels=labels)

print(job_desc[['Experience Required', 'Normalized Experience', 'Experience Category']])

    Experience Required  Normalized Experience Experience Category
0             3-5 years                    4.0         3 - 5 Years
1                  None                    NaN                 NaN
2                  None                    NaN                 NaN
3                  None                    NaN                 NaN
4                  None                    NaN                 NaN
..                  ...                    ...                 ...
123                None                    NaN                 NaN
124             1 years                    1.0              1 Year
125                None                    NaN                 NaN
126            28 years                   28.0            5+ Years
127             2 years                    2.0         1 - 2 Years

[128 rows x 3 columns]


In [29]:
job_desc['Experience Category']

0      3 - 5 Years
1              NaN
2              NaN
3              NaN
4              NaN
          ...     
123            NaN
124         1 Year
125            NaN
126       5+ Years
127    1 - 2 Years
Name: Experience Category, Length: 128, dtype: category
Categories (6, object): ['Fresh Graduate' < '1 Year' < '1 - 2 Years' < '2 - 3 Years' < '3 - 5 Years' < '5+ Years']

In [30]:
job_desc['Experience Category'].value_counts()

1 - 2 Years       19
5+ Years          18
3 - 5 Years       16
2 - 3 Years       13
1 Year            12
Fresh Graduate     4
Name: Experience Category, dtype: int64

In [31]:
job_desc['Experience Category'] = job_desc['Experience Category'].astype(object)

# Replace NaN values with "Not Stated"
job_desc['Experience Category'] = job_desc['Experience Category'].fillna('Not Stated Specifically in Years')

In [32]:
job_desc['Experience Category'].value_counts()

Not Stated Specifically in Years    46
1 - 2 Years                         19
5+ Years                            18
3 - 5 Years                         16
2 - 3 Years                         13
1 Year                              12
Fresh Graduate                       4
Name: Experience Category, dtype: int64

In [33]:
job_desc[(job_desc['Experience Category'] == "Fresh Graduate")]

,Title,Company,Description,B-HSkill,B-SSkill,B-Tech,Experience Required,Normalized Experience,Experience Category
23,Subsurface Data Scientist Intern,PT Sorik Marapi Geothermal Power,Job Specification:: Job Specification: Assis...,"[database software related, geophysics, job, e...",[problem solving],"[geology, microsoft]",Fresh Graduate,0.0,Fresh Graduate
24,Data Engineer,PT Multipolar Technology Tbk,Scope of Work:: Requirements:: Scope of Work: ...,"[information technology, computer, database, g...",[self],"[cloudera, hadoop, db2, etl, microsoft ssis, t...",Fresh Graduate,0.0,Fresh Graduate
48,Marketing Insight Intern,Pengiklan Anonim,PT. Pharos Indonesia: Marketing Insight Intern...,"[business, description marketing data data pro...",0,"[excel, python, spss]",Fresh Graduate,0.0,Fresh Graduate
104,Data Management Specialist,PT Andika Mitra Sejati,Requirement: Tugas dan tanggung jawab: D3/ S1...,0,"[bahasa inggris, berkomunikasi, administrasi, ...",0,Fresh Graduate,0.0,Fresh Graduate


In [34]:
#job_desc.to_excel('./Data/skill and experience.xlsx', index=False) 

In [39]:
exp_category = pd.DataFrame(job_desc['Experience Category'].unique())

In [45]:
unique_experience = job_desc['Experience Category'].unique()

In [48]:
experience_weights = {
    'Fresh Graduate': 0,
    '1 Year': 1,
    '1 - 2 Years': 1.5,
    '2 - 3 Years': 2,
    '3 - 5 Years': 3,
    '5+ Years': 4
}

# Step 3: Create a new DataFrame with unique experience categories and their weights
exp_category = pd.DataFrame({
    'Experience Category': unique_experience,
    'Weight': [experience_weights.get(exp, None) for exp in unique_experience]
}).sort_values('Weight')

# Display the new DataFrame


In [51]:
exp_category.to_csv('./Data/exp_category.csv')